* オムロン 環境センサ 2JCIE-BU01 のサンプルプログラム
    * オムロン社から提供していただいたサンプルプログラムをベースに作成しています    
* Bluetooth 接続する前のセンサから送信されるアドバタイズ（ブロードキャスト通信）を受信して表示


In [1]:
import sys
import time
import asyncio
from datetime import datetime
from bleak import BleakScanner

def bytetoint(buf, sig):
    """
    bytes data retrieved from 2JCIE-BU01 to integer.
    """
    return int.from_bytes(buf, byteorder='little', signed=sig)


In [2]:
def advtype01(data, output_mode):
    """
    print advertising packet : data type 1 (= sensor data)
    """
    current_time        = str(datetime.now())
    sequence_number     = str(bytetoint(data[1:2], False))
    temparature         = str(bytetoint(data[2:4], True) / 100)
    relative_humidity   = str(bytetoint(data[4:6], False) / 100)
    ambient_light       = str(bytetoint(data[6:8], False))
    barometric_pressure = str(bytetoint(data[8:12], False) / 1000)
    sound_noise         = str(bytetoint(data[12:14], False) / 100)
    eTVOC               = str(bytetoint(data[14:16], False))
    eCO2                = str(bytetoint(data[16:18], False))
    if output_mode == 'CSV':
        print(current_time + ",01," + sequence_number + "," + temparature + "," + relative_humidity + "," + ambient_light + "," + barometric_pressure + "," + sound_noise + "," + eTVOC + "," + eCO2)
    else:
        print("Data Type: 01", current_time)
        print("Sequence number: " + sequence_number)
        print("Temparature [degreeC]: " + temparature)
        print("Relative humidity [%RH]: " + relative_humidity)
        print("Ambient light [lx]: " + ambient_light)
        print("Barometric pressure [hPa]: " + barometric_pressure)
        print("Sound noise (LA) [dB]: " + sound_noise)
        print("eTVOC [ppb]: " + eTVOC)
        print("eCO2 [ppm]: " + eCO2)


def advtype02(data, output_mode):
    """
    print advertising packet : data type 2 (= calcuration data)
    """
    current_time        = str(datetime.now())
    sequence_number     = str(bytetoint(data[1:2], False))
    discomfort_index    = str(bytetoint(data[2:4], False) / 100)
    heat_stroke         = str(bytetoint(data[4:6], True) / 100)
    vibration_inform    = str(bytetoint(data[6:7], False))
    SI_value            = str(bytetoint(data[7:9], False) / 10)
    PGA                 = str(bytetoint(data[9:11], False) / 10)
    Seismic_intensity   = str(bytetoint(data[11:13], False) / 1000)
    acceleration_x      = str(bytetoint(data[13:15], True) / 10)
    acceleration_y      = str(bytetoint(data[15:17], True) / 10)
    acceleration_z      = str(bytetoint(data[17:19], True) / 10)
    if output_mode == 'CSV':
        print(current_time + ",02," + sequence_number + "," + discomfort_index + "," + heat_stroke + "," + vibration_inform + "," + SI_value + "," + PGA + "," + Seismic_intensity + "," + acceleration_x + "," + acceleration_y + "," + acceleration_z)
    else:
        print("Data Type: 02", current_time)
        print("Sequence number: " + sequence_number)
        print("Discomfort index: " + discomfort_index)
        print("Heat stroke [degreeC]: " + heat_stroke)
        print("Vibration information: " + vibration_inform)
        print("SI value [kine]: " + SI_value)
        print("PGA [gal]: " + PGA)
        print("Seismic intensity: " + Seismic_intensity)
        print("Acceleration X-dir. [gal]: " + acceleration_x)
        print("Acceleration Y-dir. [gal]: " + acceleration_y)
        print("Acceleration Z-dir. [gal]: " + acceleration_z)


In [4]:

def advcallback(dev, advdata):
    """
    discrimination of advertising mode  
    """
    output_mode = 'CSV'
    if ( dev.name == 'Rbt' ) :
        if ( 0x02D5 in advdata.manufacturer_data.keys() ): #0x02D5=companyID
            if output_mode == 'CSV':
                print(dev.address, end=",")
            else:
                print("Address: " + dev.address)
            data = advdata.manufacturer_data[0x02D5]
            if data[0] == 0x01: # mode 1
                advtype01(data, output_mode)
            elif data[0] == 0x02: # mode 2
                advtype02(data, output_mode)
            else:
                print("")
                print("unknown: " + str(data))


async def run():
    """
    scan bluetooth devices
    """
    for i in range(10):
        devices = await BleakScanner.discover(timeout=1.0, scanning_mode='active', detection_callback=advcallback)

await run()

F9:E9:A4:FB:96:C9,2024-05-01 14:26:09.603615,01,105,22.54,72.66,75,1005.48,55.01,7,446
F9:E9:A4:FB:96:C9,2024-05-01 14:26:09.606286,01,105,22.54,72.66,75,1005.48,55.01,7,446
F9:E9:A4:FB:96:C9,2024-05-01 14:26:18.409051,01,114,22.49,72.97,75,1005.467,41.09,6,445
F9:E9:A4:FB:96:C9,2024-05-01 14:26:18.410061,01,114,22.49,72.97,75,1005.467,41.09,6,445
